# Quantify and Correct SNIa Forced-Flux Lightcurves for Template Contamination

Contact author: Melissa Graham

Date last verified to run: Thu Dec 29, 2022

RSP environment version: Weekly 2022_40


## 1. Introduction

For DP0.2, the template images were constructed using the 30% of the images with the best seeing over the 6-year DC2 simulation.

This results in some flux in the template images ("template contamination") for some supernovae, which is then subtracted during difference image analysis, leading to negative or reduced difference-image fluxes.

**Quantify template contamination:** Section 3 estimates that 50% of SNIa lightcurves are affected by template contamination.

**Correct SNIa lightcurves for template contamination:** Section 4 demonstrates a method for correcting for template contamination using the forced photometry fluxes during "off-peak" visits obtained before and after then supernova.


### 1.1. Import packages

In [ ]:
import time
import numpy as np
import pandas
import matplotlib
import matplotlib.pyplot as plt

from astropy.units import UnitsWarning
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
import astropy.units as u

from lsst.rsp import get_tap_service
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler
import lsst.geom as geom

### 1.2. Set global parameters and functions

In [ ]:
service = get_tap_service()

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')
pandas.set_option('display.max_rows', 1000)

In [ ]:
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u' : '#56b4e9', 'g' : '#008060', 'r' : '#ff4000',
                      'i' : '#850000', 'z' : '#6600cc', 'y' : '#000000'}
plot_filter_symbols = {'u' : 'o', 'g' : '^', 'r' : 'v', 
                       'i' : 's', 'z' : '*', 'y' : 'p'}

## 2. Establish a sample of SNIa

### 2.1. Get a sample of z<0.5 true SNIa

Use a radius = 4 degree cone search near the center of DC2.

This should return 10407 true SNIa and take < 2 minutes.

In [ ]:
%%time

# query = "SELECT mt.id_truth_type, ts.ra, ts.dec, ts.truth_type "\
#         "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
#         "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
#         "WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 57.5, -36.5, 4)) = 1 "\
#         "AND ts.truth_type = 3 AND ts.redshift < 0.5"

query = "SELECT ra, dec, truth_type, redshift "\
        "FROM dp02_dc2_catalogs.TruthSummary "\
        "WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 57.5, -36.5, 4)) = 1 "\
        "AND truth_type = 3 AND redshift < 0.5"
results = service.search(query)
del query

Convert results to a pandas table with `to_table().to_pandas()`.

In [ ]:
TrueSNIa = results.to_table().to_pandas()
del results
print('len(TrueSNIa) = ', len(TrueSNIa))

Option to display table contents.

In [ ]:
# TrueSNIa

### 2.2. Get a sample of `DiaObjects`

Retrieve DiaObjects from the same region which were detected with a maximum flux of at least 1000 nJy in the r- and i-bands, and at least 5 detections in r- and i-bands.

SNIa that have a redshift < 0.5 and were detected near peak brightness would meet this criteria.

This should return 116953 `DiaObjects` and take < 1 minute.

Convert results to a pandas table with `to_table().to_pandas()`.

In [ ]:
%%time

query = "SELECT diaObjectId, ra, decl "\
        "FROM dp02_dc2_catalogs.DiaObject "\
        "WHERE CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 57.5, -36.5, 4)) = 1 "\
        "AND rPSFluxMax > 1000 AND iPSFluxMax > 1000 "\
        "AND rPSFluxNdata > 5 AND iPSFluxNdata > 5"
results = service.search(query)
del query

Convert results to a pandas table with `to_table().to_pandas()`.

In [ ]:
DiaObjs = results.to_table().to_pandas()
del results
print('len(DiaObjs) = ', len(DiaObjs))

Option to display table contents.

In [ ]:
# DiaObjs

### 2.3. Match `TrueSNIa` to `DiaObjs`

First, use `astropy` to create arrays of sky coordinates (RA, Dec) using `SkyCoord`.

In [ ]:
TrueSNIa_skycoord = SkyCoord(ra=TrueSNIa.loc[:, 'ra'].values*u.degree, 
                             dec=TrueSNIa.loc[:, 'dec'].values*u.degree)
DiaObjs_skycoord = SkyCoord(ra=DiaObjs.loc[:, 'ra'].values*u.degree, 
                            dec=DiaObjs.loc[:, 'decl'].values*u.degree)

Use `match_coordinates_sky` to find the nearest `DiaObjs` table entry for each `TrueSNIa` table entry.

`idx` = the index in `DiaObjs` to the nearest match of each `TrueSNIa`

`d2d` = the two-dimenstional distance to the nearest match, in arcsec

`d3d` = a three-dimensional distance is returned, but meaningless for our use-case

In [ ]:
idx, d2d, d3d = match_coordinates_sky(TrueSNIa_skycoord, DiaObjs_skycoord)

In the figure below, the distribution of 2d distances to the nearest `DiaObjs` for each `TrueSNIa` shows that applying a maximum of 1 arcsecond as a "real" match would be appropriate.

In [ ]:
fig = plt.figure(figsize=(4, 2))
plt.hist(np.log10(d2d.arcsec), bins=50)
plt.xlabel('log distance to nearest match [arcsec]')
plt.show()

Print the number of `TrueSNIa` that are matched within 1 arcsecond of a `DiaObjs`.

This will show there are 847 `DiaObjs` with which to explore template contamination.

In [ ]:
tx = np.where(d2d.arcsec < 1.0)[0]
print(len(tx))
del tx

Add columns to the `TrueSNIa` table to store the match information for good matches.

Use negative values as the placeholder.

In [ ]:
TrueSNIa['DiaObjs_idx'] = np.zeros(len(TrueSNIa), dtype='int') - 1
TrueSNIa['DiaObjs_d2d'] = np.zeros(len(TrueSNIa), dtype='float') - 1.0

Fill in the database with match information for the good matches only.

In [ ]:
d2d_arcsec = d2d.arcsec
tx = np.where(d2d.arcsec < 1.0)[0]
for x in tx:
    TrueSNIa.loc[x, 'DiaObjs_idx'] = idx[x]
    TrueSNIa.loc[x, 'DiaObjs_d2d'] = d2d_arcsec[x]
del tx, d2d_arcsec
del idx, d2d, d3d

Option to see how the two columns have been added to the table, with negative values used as placeholders for no match information.

In [ ]:
# TrueSNIa

### 2.4. Make the list of `DiaObjects` that are SNIa

Create a string, formatted as a tuple, that contains a list of all the `diaObjectId` values for the elements of `DiaObjs` matched to `TrueSNIa`.

In [ ]:
tuple_string_diaObjectId = '('
tx = np.where(TrueSNIa.loc[:, 'DiaObjs_idx'] >= 0)[0]
for i, x in enumerate(tx):
    tuple_string_diaObjectId += str(DiaObjs.loc[TrueSNIa.loc[x, 'DiaObjs_idx'], 'diaObjectId'])
    if i < len(tx)-1:
        tuple_string_diaObjectId += ', '
    else:
        tuple_string_diaObjectId += ')'
del tx

Option to view the string-formatted tuple.

In [ ]:
# print(tuple_string_diaObjectId)

After this, we don't use the DiaObjs or TrueSNIa dataframes again.

In [ ]:
del DiaObjs, TrueSNIa

## 3. Estimate the fraction of SNIa affected by template contamination

Without template contamination, supernovae would never be detected with a negative flux value in a difference image.

In other words, the minimum difference-image fluxes for `DiaObjects` matched with true SNIa _should_ always be positive.

The fraction which are negative indicates approximately how many true SNIa `DiaObjects` are impacted by template contamination.

Use the forced photometry in the `ForcedSourceOnDiaObjects` table to determin how often the true SNIa `DiaObjects` identified above are detected with negative flux during visits that are well before or well after ("off-peak") the SNIa event.

### 3.1. Retrieve difference-image forced photometry

The PSF-fit fluxes are available in `ForcedSourceOnDiaObjects`, but to get the visit dates requires a join with the `CcdVisit` table.

Retrieve data from `ForcedSourceOnDiaObjects` for all forced sources associated with a `DiaObject` which was matched to a true SNIa.

Retrive also the `coord_ra` and `coord_dec` from `ForcedSourceOnDiaObjects`, and the `visit` and `detector` from `CcdVisit`, for use in Section 5.

In [ ]:
%%time

query = "SELECT fsodo.diaObjectId, fsodo.ccdVisitId, fsodo.coord_ra, fsodo.coord_dec, "\
        "fsodo.psfDiffFlux, fsodo.psfDiffFluxErr, fsodo.psfDiffFlux_flag, "\
        "ccdv.ccdVisitId, ccdv.visitId, ccdv.detector, ccdv.band, ccdv.expMidptMJD "\
        "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject AS fsodo "\
        "JOIN dp02_dc2_catalogs.CcdVisit AS ccdv ON ccdv.ccdVisitId = fsodo.ccdVisitId "\
        "WHERE fsodo.diaObjectId IN "+tuple_string_diaObjectId
results = service.search(query)
del query

Convert results to a pandas table.

In [ ]:
FSrc = results.to_table().to_pandas()
del results

Option to view the table of data retrieved.

In [ ]:
# FSrc

### 3.2. Identify dates of peak brightess 

Make an array of the unique `diaObjectid` for the `DiaObjects` matched to true SNIa, for which forced photometry was retrieved from the `ForcedSourceOnDiaObject` table.

In [ ]:
SNIa_diaObjectId = np.unique(FSrc.loc[:, 'diaObjectId'])

Determine the date of the brightest detection ("`SNIa_bdet_date`") in MJD.

This is not necessarily the date of peak light (maximum brightness) for SNIa, which is typically determined with multi-band lightcurve fits.
But it is close enough for the purposes of this demonstration.

In [ ]:
%%time

SNIa_bdet_date = np.zeros(len(SNIa_diaObjectId), dtype='float')
for i, id_val in enumerate(SNIa_diaObjectId):
    sx = np.where(FSrc['diaObjectId'] == id_val)[0]
    mx = np.argmax(FSrc.loc[sx, 'psfDiffFlux'])
    SNIa_bdet_date[i] = FSrc.loc[sx[mx], 'expMidptMJD']
    del sx, mx

### 3.3. Calculate the off-peak flux statistics

Make arrays of the minimum, maximum, mean, and standard deviation (`std`) of the difference-image forced flux in the "off-peak" visits (i.e., >300 days before, or >500 days after, the peak date), along with the number of visits (`N`) that contributed to these statistical measures, for each of the six filters.

Only use unflagged values of `psfDiffFlux` in these statistical measures.

This takes about 30 seconds to compute.

In [ ]:
%%time

SNIa_offpeak_min = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_max = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_mean = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_std = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_N = np.zeros((len(SNIa_diaObjectId), 6), dtype='int')

for i, id_val in enumerate(SNIa_diaObjectId):
    sx = np.where(FSrc['diaObjectId'] == id_val)[0]
    if len(sx) > 0:
        for f, filt in enumerate(plot_filter_labels):
            fx = np.where(FSrc.loc[sx, 'band'] == filt)[0]
            if len(fx) > 0:
                dx = np.where((FSrc.loc[sx[fx], 'psfDiffFlux_flag'] == 0)
                              & ((FSrc.loc[sx[fx], 'expMidptMJD'] <= SNIa_bdet_date[i]-300) 
                                 |(FSrc.loc[sx[fx], 'expMidptMJD'] >= SNIa_bdet_date[i]+500)))[0]
                SNIa_offpeak_N[i, f] = len(dx)
                if len(dx) > 0:
                    SNIa_offpeak_min[i, f] = np.min(FSrc.loc[sx[fx[dx]], 'psfDiffFlux'])
                    SNIa_offpeak_max[i, f] = np.max(FSrc.loc[sx[fx[dx]], 'psfDiffFlux'])
                    SNIa_offpeak_mean[i, f] = np.mean(FSrc.loc[sx[fx[dx]], 'psfDiffFlux'])
                    SNIa_offpeak_std[i, f] = np.std(FSrc.loc[sx[fx[dx]], 'psfDiffFlux'])
                del dx
            del fx
    del sx

### 3.4. Plots to explore the off-peak forced flux statistics

#### 3.4.1. Number of off-peak visits

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=False)

f = 0
for i in range(3):
    for j in range(2):
        filt = plot_filter_labels[f]
        ax[i,j].hist(SNIa_offpeak_N[:, f], bins=20, histtype='step', log=True,
                     lw=1, ls='solid', color=plot_filter_colors[filt])
        f += 1

ax[2,0].set_xlabel('# Off-peak Visits')
ax[2,1].set_xlabel('# Off-peak Visits')
ax[0,0].set_ylabel('# SNIa')
ax[1,0].set_ylabel('# SNIa')
ax[2,0].set_ylabel('# SNIa')
plt.show()

#### 3.4.2. Minimum, maximum, and mean of off-peak forced fluxes

Plot the distribution of minimum (dotted), maximum (dashed), and mean (solid) difference-image forced photometry fluxes for the `DiaObjects` that are matched to a true SNIa.

Notice that the x-axis limits for the 'y' band are broader.

Grey lines indicate the _approximate_ 5-sigma detection limit in the processed visit images (PVI; direct images).

`depths_PVI_mags` = the approximate 5-sigma limiting magnitudes for a single direct image, in magnitudes

`SNR5_PVI_nJy` = same as `depths_PVI_mags`, but in flux.

When the off-peak mean forced flux value is less than the 5-sigma limit, it can be considered a significant negative detection in the difference image.

In [ ]:
depths_PVI_mags = np.asarray([23.9,25.0,24.7,24.0,23.3,22.1], dtype='float')
SNR5_PVI_nJy = 10.0**((depths_PVI_mags - 31.4) / -2.5)

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=False)

f = 0
for i in range(3):
    for j in range(2):
        filt = plot_filter_labels[f]
        
        ax[i,j].axvspan(-1.0 * SNR5_PVI_nJy[f], SNR5_PVI_nJy[f], alpha=0.2, 
                        color='grey', label='SNR<5')
        ax[i,j].hist(SNIa_offpeak_min[:, f], bins=150, histtype='step', log=True,
                     lw=1, ls='dotted', color=plot_filter_colors[filt], label='min')
        ax[i,j].hist(SNIa_offpeak_max[:, f], bins=150, histtype='step', log=True,
                     lw=1, ls='dashed', color=plot_filter_colors[filt], label='max')
        ax[i,j].hist(SNIa_offpeak_mean[:, f], bins=150, histtype='step', log=True,
                     lw=2, ls='solid', color=plot_filter_colors[filt], label='mean')

        if filt == 'u': ax[i,j].set_xlim([-8000, 2000])
        elif filt == 'g': ax[i,j].set_xlim([-10000, 3000])
        elif filt == 'r': ax[i,j].set_xlim([-20000, 5000])
        elif filt == 'i': ax[i,j].set_xlim([-20000, 5000])
        elif filt == 'z': ax[i,j].set_xlim([-20000, 5000])
        elif filt == 'y': ax[i,j].set_xlim([-20000, 5000])

        ax[i,j].legend(loc='upper left')
        f += 1

ax[2,0].set_xlabel('Off-peak Forced Flux [nJy]')
ax[2,1].set_xlabel('Off-peak Forced Flux [nJy]')
ax[0,0].set_ylabel('# SNIa')
ax[1,0].set_ylabel('# SNIa')
ax[2,0].set_ylabel('# SNIa')
plt.show()

#### 3.4.3. Mean vs. standard deviation in off-peak forced fluxes

The plot below visualizes the standard deviation about the mean of the off-peak forced fluxes.

Where the standard deviation is low, the mean is an accurate estimate of the off-peak flux.

The plot shows that in most cases where the off-peak forced flux is "significantly negative", and detected with SNR>5 (outside the grey region), the standard deviation is consistently low (<500 nJy).

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=False)

f = 0
for i in range(3):
    for j in range(2):
        filt = plot_filter_labels[f]
        
        ax[i,j].axvspan(-1.0 * SNR5_PVI_nJy[f], SNR5_PVI_nJy[f], alpha=0.2, 
                        color='grey', label='SNR<5')
        ax[i,j].plot(SNIa_offpeak_mean[:, f], SNIa_offpeak_std[:, f],
                     plot_filter_symbols[filt], color=plot_filter_colors[filt])

        if filt == 'u': ax[i,j].set_xlim([-8000, 1000])
        elif filt == 'g': ax[i,j].set_xlim([-10000, 2000])
        elif filt == 'r': ax[i,j].set_xlim([-20000, 2000])
        elif filt == 'i': ax[i,j].set_xlim([-20000, 2000])
        elif filt == 'z': ax[i,j].set_xlim([-20000, 2000])
        elif filt == 'y': ax[i,j].set_xlim([-20000, 2000])
        
        ax[i,j].set_ylim([0,2000])
        if filt == 'y': ax[i,j].set_ylim([0,4000])

        # ax[i,j].legend(loc='upper left')
        f += 1

ax[2,0].set_xlabel('Mean Off-Peak Forced Flux [nJy]')
ax[2,1].set_xlabel('Mean Off-Peak Forced Flux [nJy]')
ax[0,0].set_ylabel('Std. Dev. [nJy]')
ax[1,0].set_ylabel('Std. Dev. [nJy]')
ax[2,0].set_ylabel('Std. Dev. [nJy]')
plt.show()

### 3.5. Estimate the fraction of true SNIa with "significantly negative" off-peak difference-image forced fluxes

Calculate the fraction of SNIa that have a mean forced flux during "off-peak" epochs that is "significantly negative" (i.e., with SNR > 5), for each filter.
This is approximately the fraction of true SNIa affected by template contamination, per filter.

In [ ]:
frac_filt = np.zeros(6, dtype='float')
for f, filt in enumerate(plot_filter_labels):
    fx = np.where(np.isfinite(SNIa_offpeak_mean[:, f]))[0]
    tx = np.where(SNIa_offpeak_mean[fx, f] < -1.0 * SNR5_PVI_nJy[f])[0]
    frac_filt[f] = np.round(float(len(tx)) / float(len(fx)), 3)
    print(filt, frac_filt[f])
    del fx, tx
del frac_filt

Calculate the fraction affected _in any filter_. 

In [ ]:
tempN = len(SNIa_offpeak_mean[:, f])
tempflag1 = np.zeros(tempN, dtype='int')
tempflag2 = np.zeros(tempN, dtype='int')

for i in range(tempN):
    for f, filt in enumerate(plot_filter_labels):
        if np.isfinite(SNIa_offpeak_mean[i, f]):
            tempflag1[i] = 1
            if SNIa_offpeak_mean[i, f] < -1.0 * SNR5_PVI_nJy[f]:
                tempflag2[i] = 1

tx1 = np.where(tempflag1 == 1)[0]
tx2 = np.where(tempflag2 == 1)[0]
print(tempN, len(tx1), len(tx2))
print(np.round(float(len(tx2)) / float(len(tx1)), 3))
del tempN, tempflag1, tempflag2, tx1, tx2

This suggests that 50% of all SNIa are affected by template contamination in at least one band.

Since the templates are generated from the third of images with the best seeing, it makes sense that SNIa would experience template contamination about a third of the time in a given filter. 

The reason why the _gri_ filters are affected more than _uzy_ is likely due to detectablity -- SNIa are fainter in _uzy_, and spend less time above the magnitude threshold, leading to fewer cases of template contamination which is also a smaller effect, less detectable by our brute method, when it does occur.

## 4. Correct a lightcurve for template contamination

### 4.1. Identify a SNIa to use for this demo

Identify a few SNIa that seem particularly badly affected by template contamination.

In [ ]:
tx = np.where((SNIa_offpeak_mean[:, 1] < -1.0 * SNR5_PVI_nJy[1])
             & (SNIa_offpeak_mean[:, 2] < -1.0 * SNR5_PVI_nJy[2])
             & (SNIa_offpeak_mean[:, 3] < -1.0 * SNR5_PVI_nJy[3])
             & (SNIa_offpeak_std[:, 1] < 500)
             & (SNIa_offpeak_std[:, 2] < 500)
             & (SNIa_offpeak_std[:, 3] < 500)
             & (SNIa_offpeak_N[:, 1] > 30)
             & (SNIa_offpeak_N[:, 2] > 30)
             & (SNIa_offpeak_N[:, 3] > 30))[0]
for x in tx:
    print('%21s  %8.1f %8.1f %8.1f  %7.1f %7.1f %7.1f  %3i %3i %3i' % 
          (SNIa_diaObjectId[x], SNIa_offpeak_mean[x, 1], 
           SNIa_offpeak_mean[x, 2], SNIa_offpeak_mean[x, 3],
           SNIa_offpeak_std[x, 1], SNIa_offpeak_std[x, 2],
           SNIa_offpeak_std[x, 3], SNIa_offpeak_N[x, 3],
           SNIa_offpeak_N[x, 1], SNIa_offpeak_N[x, 2]))
del tx

Choose one of the above to explore in more detail.

In [ ]:
use_id = 1822057892992253972

In [ ]:
snx = np.where(SNIa_diaObjectId == use_id)[0]
fsx = np.where(FSrc['diaObjectId'] == use_id)[0]

### 4.2. Plot the forced flux lightcurve

Also plot the off-peak statistical measures of mean forced flux (dashed line) and a grey area marking +/- one standard deviation about the mean.

The particular SNIa chosen, with `diaObjectId` = `1822057892992253972`, occurred near the end of the DC2 simulation and thus there are a great many off-peak visits to use to estimate the template contamination.

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=True)
f = 0
for i in range(3):
    for j in range(2):
        filt = plot_filter_labels[f]
        
        ax[i,j].axhline(0.0, lw=1, ls='solid', alpha=1, color='black')
        ax[i,j].axhline(SNIa_offpeak_mean[snx, f], lw=2, ls='dashed', alpha=1, color='black')
        templo = float(SNIa_offpeak_mean[snx, f] - SNIa_offpeak_std[snx, f])
        temphi = float(SNIa_offpeak_mean[snx, f] + SNIa_offpeak_std[snx, f])
        ax[i,j].axhspan(templo, temphi, alpha=0.4, color='grey')
        del templo, temphi
        
        fx = np.where((FSrc.loc[fsx, 'band'] == filt) 
                      & (np.isfinite(FSrc.loc[fsx, 'psfDiffFlux'])))[0]
        xvals = FSrc.loc[fsx[fx], 'expMidptMJD']
        yvals = FSrc.loc[fsx[fx], 'psfDiffFlux']
        yevals = FSrc.loc[fsx[fx], 'psfDiffFluxErr']
        ax[i,j].errorbar(xvals, yvals, yevals, fmt=plot_filter_symbols[filt],
                 color=plot_filter_colors[filt])
        del fx, xvals, yvals, yevals, filt
        
        f += 1

ax[2,0].set_xlabel('Modified Julian Date')
ax[2,1].set_xlabel('Modified Julian Date')
ax[0,0].set_ylabel('Forced Flux [nJy]')
ax[1,0].set_ylabel('Forced Flux [nJy]')
ax[2,0].set_ylabel('Forced Flux [nJy]')
plt.show()

### 4.3. Apply the flux correction and re-plot

To correct the forced flux values, subtract the value of `SNIa_offpeak_mean`.

To include this correction in the error bars, add in quadrature the forced flux error and the value of `SNIa_offpeak_std`.

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=True)
f = 0
for i in range(3):
    for j in range(2):
        filt = plot_filter_labels[f]

        ax[i,j].axhline(0.0, lw=1, ls='solid', alpha=1, color='black')
        
        fx = np.where((FSrc.loc[fsx, 'band'] == filt) 
                      & (np.isfinite(FSrc.loc[fsx, 'psfDiffFlux'])))[0]
        xvals = FSrc.loc[fsx[fx], 'expMidptMJD']
        yvals = FSrc.loc[fsx[fx], 'psfDiffFlux'] - SNIa_offpeak_mean[snx, f]
        yevals = np.sqrt((FSrc.loc[fsx[fx], 'psfDiffFluxErr']**2)
                         + (SNIa_offpeak_std[snx, f]**2))
        ax[i,j].errorbar(xvals, yvals, yevals, fmt=plot_filter_symbols[filt],
                 color=plot_filter_colors[filt])
        del fx, xvals, yvals, yevals, filt

        f += 1

ax[0,0].set_xlim([60750, 61500])
ax[2,0].set_xlabel('Modified Julian Date')
ax[2,1].set_xlabel('Modified Julian Date')
ax[0,0].set_ylabel('Corrected Flux [nJy]')
ax[1,0].set_ylabel('Corrected Flux [nJy]')
ax[2,0].set_ylabel('Corrected Flux [nJy]')
plt.show()

## 5. Review images exhibiting template contamination

Use the `butler` for image retrieval, and `afw_display` for image display.

### 5.1. Define a cutout function

Although it is best practice to define all functions at the top of a notebook, since `cutout_image()` is only used here in Section 5 we define it just now.

In [ ]:
def cutout_image(butler, ra, dec, visit, detector, 
                 imageType='goodSeeingDiff_templateExp', 
                 cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a calexp at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    visit: int
        Visit id
    detector: int
        Detector id
    imageType: str
        Either 'goodSeeingDiff_templateExp' or 'goodSeeingDiff_differenceExp'
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)    
    image_wcs = butler.get(imageType+'.wcs', **dataId)
    xy = geom.PointI(image_wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get(imageType, parameters=parameters, **dataId)

    return cutout_image

### 5.2. Display the r-band template and difference images for 2 visits

First, we need the selected SNIa's RA and Dec.

In [ ]:
sn_ra = FSrc.loc[fsx[0], 'coord_ra']
sn_dec = FSrc.loc[fsx[0], 'coord_dec']
print(sn_ra, sn_dec)

Choose two visits from around the SNIa explosion.

In [ ]:
tx = np.where((FSrc.loc[fsx, 'band'] == 'r')
              & (FSrc.loc[fsx, 'expMidptMJD'] > 61200)
              & (FSrc.loc[fsx, 'expMidptMJD'] < 61310))[0]
for x in tx:
    print(FSrc.loc[fsx[x], 'expMidptMJD'], FSrc.loc[fsx[x], 'psfDiffFlux'],
          FSrc.loc[fsx[x], 'visitId'], FSrc.loc[fsx[x], 'detector'], )

Make an array of the `[visit, detector]` pairs for the chosen two visits.

In [ ]:
use_visit_dets = np.asarray([[1155562, 73], [1171793, 68]], dtype='int')

Use the function `cutout_image()` to create cutouts of the template and difference images for the chosen two visits, at the location of the SNIa.

In [ ]:
temp1 = cutout_image(butler, sn_ra, sn_dec, 
                     use_visit_dets[0,0], use_visit_dets[0,1], 
                     imageType='goodSeeingDiff_templateExp')
diff1 = cutout_image(butler, sn_ra, sn_dec, 
                     use_visit_dets[0,0], use_visit_dets[0,1], 
                     imageType='goodSeeingDiff_differenceExp')
temp2 = cutout_image(butler, sn_ra, sn_dec, 
                     use_visit_dets[1,0], use_visit_dets[1,1], 
                     imageType='goodSeeingDiff_templateExp')
diff2 = cutout_image(butler, sn_ra, sn_dec, 
                     use_visit_dets[1,0], use_visit_dets[1,1], 
                     imageType='goodSeeingDiff_differenceExp')

Plot the template at left, and the difference at right, for the two selected visits.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8, 7))

plt.sca(ax[0, 0])  # set the first axis as current
display1 = afwDisplay.Display(frame=fig)
display1.scale('linear', 'zscale')
display1.mtv(temp1.image)
ax[0, 0].set_title('template for visit '+str(use_visit_dets[0, 0]))

plt.sca(ax[0, 1])  # set the second axis as current
display2 = afwDisplay.Display(frame=fig)
display2.scale('linear', 'zscale')
display2.mtv(diff1.image)
ax[0, 1].set_title('difference for visit '+str(use_visit_dets[0, 0]))

plt.sca(ax[1, 0])  # set the second axis as current
display3 = afwDisplay.Display(frame=fig)
display3.scale('linear', 'zscale')
display3.mtv(temp2.image)
ax[1, 0].set_title('template for visit '+str(use_visit_dets[1, 0]))

plt.sca(ax[1, 1])  # set the second axis as current
display4 = afwDisplay.Display(frame=fig)
display4.scale('linear', 'zscale')
display4.mtv(diff2.image)
ax[1, 1].set_title('difference for visit '+str(use_visit_dets[1, 0]))

plt.tight_layout()
plt.show()